In [1]:
import pandas as pd
import os
import cv2

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
path="train/"
lst = os.listdir(path)

In [3]:
df = pd.DataFrame(lst,columns=["Images"])

In [4]:
df.head()

,Images
0,cat.0.jpg
1,cat.1.jpg
2,cat.10.jpg
3,cat.100.jpg
4,cat.1000.jpg


In [5]:
df.shape

(25000, 1)

In [6]:
# labelling data using simple for loop

In [7]:
lab = []
for i in df['Images']:
    if i.startswith("cat"):
        lab.append(1)
    else:
        lab.append(2)

In [8]:
len(lab)

25000

In [9]:
# labelling data using list comprehension

In [10]:
lab1=[1 if x.startswith("cat") else 2 for x in df["Images"] ]
len(lab1)

25000

In [11]:
# labelling data using map and lambda

In [12]:
lab2= map(lambda x:["dog","cat"][x.startswith("cat")],df["Images"])
labels= list(lab2)

In [13]:
len(labels)

25000

In [14]:
# defining data and labels

In [15]:
df['labels']=labels
df.head()

,Images,labels
0,cat.0.jpg,cat
1,cat.1.jpg,cat
2,cat.10.jpg,cat
3,cat.100.jpg,cat
4,cat.1000.jpg,cat


In [16]:
df.tail()

,Images,labels
24995,dog.9995.jpg,dog
24996,dog.9996.jpg,dog
24997,dog.9997.jpg,dog
24998,dog.9998.jpg,dog
24999,dog.9999.jpg,dog


In [17]:
# shuflling the data set

In [18]:
df = df.sample(frac=1).reset_index(drop=True)

In [19]:
df.head()

,Images,labels
0,cat.4994.jpg,cat
1,dog.8916.jpg,dog
2,dog.4291.jpg,dog
3,cat.9727.jpg,cat
4,dog.5395.jpg,dog


In [20]:
df.shape

(25000, 2)

In [21]:
train_df= df[0:20000] 
valid_df= df[20000:]

In [22]:
print(" training data shape {}".format(train_df.shape))
print(" testing data shape {}".format(valid_df.shape))


 training data shape (20000, 2)
 testing data shape (5000, 2)


In [23]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [25]:
img_width, img_height = 128, 128
batch_size = 16

In [26]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, "train/", 
    x_col='Images',
    y_col='labels',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size
)

Found 20000 validated image filenames belonging to 2 classes.


In [27]:
valid_generator = train_datagen.flow_from_dataframe(
        valid_df,"train/",
        x_col="Images",
        y_col ="labels",
        target_size= (img_width, img_height),
        class_mode='categorical',
        batch_size=batch_size

)

Found 5000 validated image filenames belonging to 2 classes.


In [28]:
# modeling cnn

In [29]:
input_shape = (img_width, img_height, 3)

In [30]:
model = Sequential()
model.add(Conv2D(32,(3, 3),activation='relu',kernel_initializer='he_uniform',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation="softmax"))

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0

In [33]:
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

In [37]:
total_train=train_df.shape[0]
total_validate=valid_df.shape[0]
epochs = 20

In [38]:
model.fit_generator(
    train_generator,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=total_validate // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
1250/1250 [==============================] - 427s 342ms/step - loss: 0.7579 - accuracy: 0.5702 - val_loss: 0.7027 - val_accuracy: 0.5980
Epoch 2/20
1250/1250 [==============================] - 415s 332ms/step - loss: 0.6107 - accuracy: 0.6604 - val_loss: 0.6004 - val_accuracy: 0.6703
Epoch 3/20
1250/1250 [==============================] - 404s 323ms/step - loss: 0.5780 - accuracy: 0.6949 - val_loss: 0.5770 - val_accuracy: 0.6839
Epoch 4/20
1250/1250 [==============================] - 404s 323ms/step - loss: 0.5390 - accuracy: 0.7341 - val_loss: 0.5271 - val_accuracy: 0.7402
Epoch 5/20
1250/1250 [==============================] - 401s 321ms/step - loss: 0.5094 - accuracy: 0.7549 - val_loss: 0.4560 - val_accuracy: 0.7841
Epoch 6/20
1250/1250 [==============================] - 401s 321ms/step - loss: 0.4810 - accuracy: 0.7766 - val_loss: 0.4243 - val_accuracy: 0.8051
Epoch 7/20
1250/1250 [==============

In [39]:
model.save('final_model.h5')